In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
%matplotlib inline




In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('/gdrive/My Drive/Colab Notebooks/amazonreviews/test.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('/gdrive/My Drive/Colab Notebooks/amazonreviews/test.ft.txt.bz2')


In [0]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)


In [0]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, random_state=57643892, test_size=0.2)
MAX_FEATURES = 12000
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
val_texts = tokenizer.texts_to_sequences(val_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)
MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)
train_texts = pad_sequences(train_texts, maxlen=MAX_LENGTH)
val_texts = pad_sequences(val_texts, maxlen=MAX_LENGTH)
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)



In [0]:
def build_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
model = build_model()
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 231)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 231, 64)           768000    
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 229, 64)           12352     
_________________________________________________________________
batch_normalization_6 (Batch (None, 229, 64)           256       
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 76, 64)            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 72, 64)            20544     
_________________________________________________________________
batch_normalization_7 (Batch (None, 72, 64)            256 

In [72]:
  History = model.fit(
    train_texts, 
    train_labels, 
    batch_size=128,
    epochs=2
     )


Train on 320000 samples
Epoch 1/2
  7040/320000 [..............................] - ETA: 5:59 - loss: 0.1507 - binary_accuracy: 0.9459

KeyboardInterrupt: ignored

In [73]:
model.evaluate(val_texts,val_labels)

80000/80000 [==============================] - 29s 357us/sample - loss: 0.1894 - binary_accuracy: 0.9271


[0.1893634833984077, 0.927125]

In [0]:
preds = model.predict(test_texts)
print('Accuracy score: {:0.4}'.format(accuracy_score(test_labels, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(test_labels, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_labels, preds)))


In [0]:
print(len(preds))
positive = 0
negative =0
for i in range(len(preds)):
  if preds[i] > 0.5:
    positive = positive +1
  else:
    negative = negative+1

print(positive)
print(negative)

400000
195363
204637


In [0]:
def rating_generate(a,b):
  total= positive+negative
  rating= positive/total * 5
  return  print(rating)
ratings= rating_generate(positive,negative)



2.4420375


AttributeError: ignored